In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Union

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, classification_report, f1_score, roc_auc_score, average_precision_score

In [15]:
onset_df_pilot = pd.read_csv('/blue/yonghui.wu/lideyi/AKI_GNN/raw_data/norm_df_pilot.csv')
print(onset_df_pilot.shape)
n_class = len(np.unique(onset_df_pilot.AKI_TARGET))

(41467, 69)


In [16]:
feature_columns = [col for col in onset_df_pilot.columns if (col != 'AKI_TARGET') and (col != 'TEST_SET')]
X_train = onset_df_pilot.loc[onset_df_pilot['TEST_SET'] == 0, feature_columns].copy(deep=True)
y_train = onset_df_pilot.loc[onset_df_pilot['TEST_SET'] == 0, 'AKI_TARGET'].copy(deep=True)
X_test = onset_df_pilot.loc[onset_df_pilot['TEST_SET'] == 1, feature_columns].copy(deep=True)
y_test = onset_df_pilot.loc[onset_df_pilot['TEST_SET'] == 1, 'AKI_TARGET'].copy(deep=True)

In [17]:
# split 20% of training set as validation set
X_train, X_val = X_train.iloc[:int(0.8 * len(X_train))], X_train.iloc[int(0.8 * len(X_train)):]
y_train, y_val = y_train.iloc[:int(0.8 * len(y_train))], y_train.iloc[int(0.8 * len(y_train)):]
assert len(X_train) + len(X_val) + len(X_test) == len(onset_df_pilot)
assert len(y_train) + len(y_val) + len(y_test) == len(onset_df_pilot)
print(X_train.shape, X_val.shape, X_test.shape)

(23485, 67) (5872, 67) (12110, 67)


In [18]:
RF = RandomForestClassifier(random_state=42)
RF_parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [19]:
def train_and_evaluate_model(model, parameters: dict, X_train: Union[np.ndarray, pd.Series], y_train: Union[np.ndarray, pd.Series],
                             X_test: Union[np.ndarray, pd.Series], y_test: Union[np.ndarray, pd.Series]):
    best_model = train_model(model, parameters, X_train, y_train)
    model_performance = test_model(best_model, X_test, y_test)
    return model_performance

In [20]:
def train_model(model, parameters: dict, X_train: Union[np.ndarray, pd.Series], y_train: Union[np.ndarray, pd.Series]):
    macro_f1_scorer = make_scorer(f1_score, average='macro') 
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=parameters,
        cv=5,
        scoring=macro_f1_scorer,
        n_jobs=-1,
        verbose=1,
        )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    return best_model

In [21]:
def test_model(best_model, X_test: Union[np.ndarray, pd.Series], y_test: Union[np.ndarray, pd.Series]):
    y_test_pred = best_model.predict(X_test)
    y_test_pred_prob = best_model.predict_proba(X_test)
    performance = individual_class_performance(y_test, y_test_pred, y_test_pred_prob)
    return performance

In [22]:
def individual_class_performance(y_test: Union[np.ndarray, pd.Series], y_test_pred: Union[np.ndarray, pd.Series], 
    y_test_pred_prob: Union[np.ndarray, pd.Series]) -> pd.DataFrame:
    
    n_class = len(np.unique(y_test))
    # Generate the classification report as a dictionary
    report_dict = classification_report(y_test, y_test_pred, output_dict=True)

    # Convert to a pandas DataFrame for better formatting
    metrics_table = pd.DataFrame(report_dict).transpose()

    # Extract only the classes (0, 1, 2, 3) and metrics
    class_metrics = metrics_table.loc[[str(c) for c in range(n_class)], ['precision', 'recall', 'f1-score']]
    
    # add AUROC and AUPRC
    AUROC = []
    AUPRC = []
    for i in range(n_class):
        y_true_binary = (y_test == i).astype(int)
        y_proba_class = y_test_pred_prob[:, i]
        # compute AUROC
        AUROC.append(roc_auc_score(y_true_binary, y_proba_class))
        # compute AUPRC
        AUPRC.append(average_precision_score(y_true_binary, y_proba_class))
        
    # add to the class_metrics DataFrame
    class_metrics['AUROC'] = AUROC
    class_metrics['AUPRC'] = AUPRC
    return class_metrics

# Main Run

In [23]:
RF_peformance = train_and_evaluate_model(RF, RF_parameters, X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/home/lideyi/conda/envs/AKI_GNN/lib/python3.9/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [24]:
print(RF_peformance)

   precision    recall  f1-score     AUROC     AUPRC
0   0.913366  0.977627  0.944405  0.914026  0.979285
1   0.537623  0.352647  0.425918  0.866828  0.481983
2   0.621359  0.204473  0.307692  0.955601  0.472790
3   0.785714  0.687500  0.733333  0.989371  0.750948
